Здесь будет производится скачивание и парсинг данных с сайта auto.ru

Задача: Оценка рыночной стоимости мотоцикла с пробегом 

Данные: https://moto.drom.ru/moskva/sale/ 

Целевая переменная: цена в рублях (!)

Признаки: 
1. Фирма изготовитель (!)
2. Пробег (!)
3. Класс 
4. Год выпуска 
5. Объем двигателя 
6. Число тактов 
7. Состояние 
8. Документы
9. Город 
10. Дата публикации объявления 

Как они будут называться в таблице, разделитель ',': 

0. price
1. manufacturer
2. mileage 
3. motorcycle_class
4. year 
5. engine_capacity
6. engine_strokes 
7. damaged (1 если требует ремонта, 0 если нет)
8. documents
9. city
10. date 

Алгоритм:
1. Скачать все страницы (есть номер страницы, если забанят - знаю, откуда продолжить)
2. Получить из них список ссылкок (просто парсинг)
3. Потом идти по каждой ссылке (есть номер ссылки, если забанят - знаю, откуда продолжить)

<b>Импорт библиотек</b>

In [2]:
import numpy as np
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

<b>Класс, описывающий мотоцикл</b>

In [50]:
class Motorcycle:
    def __init__(self, soup_motorcycle, index=None):
        self.soup = soup_motorcycle
        if (index is None):
            self.index = 0
        else:
            self.index = index
            
        self.warning_string_ = "Index: " + str(self.index) + "; Warning: "
        
    def warning_(self, string):
        print(self.warning_string_ + string)
        
    def parse_information(self):
        self.price = self.get_price()
        if (self.price is None):
            self.warning_('price not found')
            return False
        
        self.manufacturer = self.get_manufacturer()
        if (self.manufacturer is None):
            self.warning_('manufacturer not found')
            return False
        
        self.mileage = self.get_mileage()
        if (self.mileage is None):
            self.warning_('mileage not found')
            return False
        
        self.year = self.get_year()
        
        # аналогично дописать для всех фичей
        # проверка на None в этой функции для других фичей не нужна 
        # (я выделил самые главные)
        # 
        return True
    
        
    def get_price(self):
        price = self.soup.select_one('span.viewbull-summary-price__value')
        if (price is not None):
            price = price.text.strip().replace(' ', '')
        else:
            price = None
        return price
    
    
    def get_manufacturer(self):
        manufacturer = self.soup.findAll(attrs={"data-field" : "model"})
        if (len(manufacturer) > 0):
            manufacturer = manufacturer[0].text.split(' ')[0].strip().replace(' ', '')
        else:
            manufacturer = None
        return manufacturer
    
    def get_mileage(self):
        mileage = self.soup.findAll(attrs={"data-field" : "motoMileage"})
        if (len(mileage) > 0):
            mileage = mileage[0].text.strip().replace(' ', '')
        else:
            mileage = None
        return mileage
    
    def get_year(self):
        year = self.soup.findAll(attrs={"data-field" : "year"})
        if (len(year) > 0):
            year = year[0].text.strip().replace(' ', '')
        else:
            year = None
        return year
    
    def __repr__(self):
         return "Motorcycle class"
        
    def __str__(self):
        string = str(self.price) + ',' + str(self.manufacturer) + ',' + \
                    str(self.mileage) + ',' + str(self.year)
        return string

<b>Информация по подключению</b>

In [28]:
url_template = 'https://moto.drom.ru'
url_list = url_template + '/moskva/sale/?page='

In [5]:
# Выбрать нужное
# Сайт с прокси: http://spys.one/proxys/RU/

proxies = None
#proxies = { 'https': '5.53.19.82:56907' }

In [6]:
#user_agent = UserAgent()
#user_agent.update()
#headers = {'User-Agent': user_agent.chrome}

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'}

<b>Вспомогательные функции для подключения</b>

In [7]:
def is_banned(soup):
    title = soup.select_one('h1.title')
    if (title is not None):
        if (title.text == 'ой…'):
            #print(soup)
            return True
    else:
        return False

In [8]:
def is_page_found(soup):
    title = soup.select_one('div.page-title')
    if (title is not None):
        if (title.text == 'Страница не найдена'):
            return False
    else:
        return True

In [25]:
def get_links(url_page, proxies=None, headers=None, sleep=False, sleep_seconds=8):
    html_page = requests.get(url_page, proxies=proxies, headers=headers).content
    soup = BeautifulSoup(html_page, 'html.parser')
    
#     if (is_banned(soup)):
#         print("BANNED")
#         return None
    
#     if (not is_page_found(soup)):
#         print("Page not found")
#         return None
    
    if (sleep):
        time.sleep(np.random.randint(sleep_seconds))
          
    return soup.select('.bulletinLink')

In [10]:
def get_motorcycle_soup(url_motorcycle, proxies=None, headers=None, sleep=False, sleep_seconds=8):
    html_motorcycle = requests.get(url_motorcycle, proxies=proxies, headers=headers).content
    soup_motorcycle = BeautifulSoup(html_motorcycle, 'html.parser')
        
#     if (is_banned(soup_motorcycle)):
#         print("BANNED")
#         return None
    
#     if (not is_page_found(soup_motorcycle)):
#         print("Page not found")
#         return None
    
    if (sleep):
        time.sleep(np.random.randint(sleep_seconds))
        
    return soup_motorcycle

<b>Создание списка ссылок</b>

In [29]:
if (False): # чтобы не запускалось при нажатии Run all
    last_page = 11

    with open('data/current_page.txt', 'r') as file:
        current_page = int(file.readline())
        current_link = int(file.readline())

    with open('data/links.txt', 'a') as f_output:
        for page in range(current_page, last_page+1):
            links = get_links(url_list + str(page), proxies, headers, True)

            if (links is None):
                print("Error while getting motorcycle links")
                current_page = page
                print("Page: " + str(current_page))
                print("Link: " + str(current_link))
                break

            for link in links:
                print(str(current_link) + '. ' + url_template + link['href'], file=f_output)
                current_link += 1

            current_page = page + 1

    with open('data/current_page.txt', 'w') as f_output:
        print(current_page, file=f_output)
        print(current_link, file=f_output)

<b>Основной цикл (не работает пока)</b>

In [12]:
if (False): # чтобы не запускалось при нажатии Run all

    with open('data/current_parce_link.txt', 'r') as file:
        current_parce_link = int(file.readline())

    with open('data/links.txt', 'r') as links_file:
        with open('data/motorcycles.txt', 'a') as motorcycle_file:
            for i, link in enumerate(links_file):
                if (i < current_parce_link):
                    continue

                link = link[link.find('h'):]

                soup = get_motorcycle_soup(link, proxies, headers, True)
                if (soup is None):
                    print("Error while getting motorcycle information")
                    print("Link: " + str(current_parce_link))
                    break

                motorcycle = Motorcycle(soup, current_parce_link)
                if (motorcycle.parse_information()):
                    print(motorcycle, file=motorcycle_file)
                else:
                    print("Error while parsing motorcycle information")
                    print("Link: " + str(current_parce_link))

                current_parce_link += 1

    with open('data/current_parce_link.txt', 'w') as f_output:
        print(current_parce_link, file=f_output)

Все, что ниже - черновик, для того, чтобы что то отдельное потестить

Если банит, то можно скачать один раз в отдельной ячейке, и потом просто по ней делать (чтобы не перекачивать)

In [46]:
number_of_link = 3
with open('data/links.txt', 'r') as links_file:
    for i in range(number_of_link):
        link = links_file.readline()
        
    link = link[link.find('h'):]
    print("Link: " + link)
    
    soup = get_motorcycle_soup(str(link), proxies, headers)

Link: https://moto.drom.ru/moskva/sale/honda-cb1100-68192572.html



Ячейка выше - один раз запустили, скачали

Теперь тестируем парсинг снизу

In [49]:
motorcycle = Motorcycle(soup)

if (motorcycle.parse_information()):
    print(motorcycle)

665000₽,Honda,17000 км,2016
